In [1]:
import csv
import json

f1 = "bones.csv"
f2 = "regions.csv"

In [2]:
bones_data = []
with open(f1, 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    for line in reader:
        d = dict(zip(header, line))
        bones_data.append(d)
        
regions_data = []
with open(f2, 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    for line in reader:
        d = dict(zip(header, line))
        regions_data.append(d)

In [3]:
ds = ['body_regions','proximal_bones', 'distal_bones', 'adjacent_bones']
for d in bones_data:
    for k in ds:
        if d[k] == '':
            d[k] = []
        else:
            d[k] = d[k].split(',')

In [4]:
bones_data[130]

{'key': 'kHand_IntermediatePhalanx05Right',
 'id_number': '131',
 'body_regions': ['kRegion_Hand', 'kRegion_HandRight'],
 'bone_name': 'Phalanx 5',
 'side': 'right',
 'position': 'Intermediate',
 'readable_name': 'Right Intermediate Phalanx 5',
 'alternate_name': 'Little Finger',
 'proximal_bones': ['kHand_ProximalPhalanx05Right'],
 'distal_bones': ['kHand_DistalPhalanx05Right'],
 'adjacent_bones': []}

In [5]:
rds = ['adjacent_regions', 'human_skeleton']
for rd in regions_data:
    for k in rds:
        if rd[k] == '':
            rd[k] = []
        else:
            rd[k] = rd[k].split(',')

In [6]:
regions_data[0]

{'key': 'kRegion_Neck',
 'id_number': '1001',
 'region_name': 'Neck',
 'region_type': 'region',
 'side': '',
 'adjacent_regions': ['kRegion_Spine',
  'kRegion_Head',
  'kRegion_Chest',
  'kRegion_Skull',
  'kRegion_Face'],
 'human_skeleton': ['kThroat_Hyoid',
  'kVertebral_CervicalVertebrae01',
  'kVertebral_CervicalVertebrae02',
  'kVertebral_CervicalVertebrae03',
  'kVertebral_CervicalVertebrae04',
  'kVertebral_CervicalVertebrae05',
  'kVertebral_CervicalVertebrae06',
  'kVertebral_CervicalVertebrae07']}

In [7]:
nodes = bones_data + regions_data

In [8]:
keys = set()
for n in nodes:
    for k in n.keys():
        keys.add(k)
keys = list(keys)

In [9]:
fixed_nodes = []
for n in nodes:
    o = {}
    for k in keys:
        if k in n.keys():
            o[k] = n[k]
        else:
            o[k] = None
    fixed_nodes.append(o)

In [10]:
edge_keys = {
    'proximal_bones':   'distal_to',
    'adjacent_bones':   'adjacent_to',
    'distal_bones':     'proximal_to',
    'body_regions':     'part_of',
    'adjacent_regions': 'adjacent_to',
    'human_skeleton':   'includes'
}

In [11]:
lookup = {}
for n in nodes:
    lookup[n['key']] = n

In [12]:
final_data = {
    "nodes":[],
    "edges":[]
}

In [13]:
for n in nodes:
    fn = {}
    for k in n.keys():
        if k not in edge_keys.keys():
            if k == 'id_number':
                fn[k] = int(n[k])
            else:
                fn[k] = n[k]
        else:
            # dealing with edges
            for target in n[k]:
                e = {
                    "edge_type": edge_keys[k],
                    "source": n['key'],
                    "dest": target
                }
                final_data['edges'].append(e)
    final_data['nodes'].append(fn)
                

In [15]:
final_data['nodes'][140:145]

[{'key': 'kHand_DistalPhalanx02Right',
  'id_number': 141,
  'bone_name': 'Phalanx 2',
  'side': 'right',
  'position': 'Distal',
  'readable_name': 'Right Distal Phalanx 2',
  'alternate_name': 'Index Finger'},
 {'key': 'kHand_DistalPhalanx03Right',
  'id_number': 142,
  'bone_name': 'Phalanx 3',
  'side': 'right',
  'position': 'Distal',
  'readable_name': 'Right Distal Phalanx 3',
  'alternate_name': 'Middle Finger'},
 {'key': 'kHand_DistalPhalanx04Right',
  'id_number': 143,
  'bone_name': 'Phalanx 4',
  'side': 'right',
  'position': 'Distal',
  'readable_name': 'Right Distal Phalanx 4',
  'alternate_name': 'Ring Finger'},
 {'key': 'kHand_DistalPhalanx05Right',
  'id_number': 144,
  'bone_name': 'Phalanx 5',
  'side': 'right',
  'position': 'Distal',
  'readable_name': 'Right Distal Phalanx 5',
  'alternate_name': 'Little Finger'},
 {'key': 'kPelvis_InnominateBoneLeft',
  'id_number': 145,
  'bone_name': 'Innominate',
  'side': 'left',
  'position': '',
  'readable_name': 'Left In

In [19]:
with open('skeleton.json','w') as output:
    json.dump(final_data, output)